In [ ]:
!pip install ase
!pip install fairchem-core
!pip install huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 51.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.6/55.6 kB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.9/284.9 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.9/42.9 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 448.0/448.0 kB 31.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 93.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 96.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 98.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 88.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 59.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━

In [ ]:
from ase import Atoms
from ase.build import bulk
from ase.constraints import ExpCellFilter
from ase.optimize import BFGS
from huggingface_hub import login

from fairchem.core import pretrained_mlip, FAIRChemCalculator

# Logging into the UMA repository
hf_token = <TOKEN>
login(token = hf_token)

device = "cpu"
predictor = pretrained_mlip.get_predict_unit("uma-s-1", device=device)
calc = FAIRChemCalculator(predictor, task_name="omat")

atoms = bulk("Cu", "fcc", a = 3.56, cubic = True)
atoms.calc = calc

opt_geo = BFGS(atoms)
opt_geo.run(fmax = 0.01)

cell_filter = ExpCellFilter(atoms, mask=[1, 1, 1, 0, 0, 0])
opt = BFGS(cell_filter)
opt.run(fmax=0.01)

print("Final lattice:\n", atoms.get_cell())
print("Final positions:\n", atoms.get_positions())

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


checkpoints/uma-s-1.pt:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

      Step     Time          Energy          fmax
BFGS:    0 13:27:43      -14.995555        0.000000
      Step     Time          Energy          fmax
BFGS:    0 13:27:43      -14.995555        2.065551


<ipython-input-3-1017426055>:23: FutureWarning: Import ExpCellFilter from ase.filters
  cell_filter = ExpCellFilter(atoms, mask=[1, 1, 1, 0, 0, 0])


BFGS:    1 13:27:44      -14.995581        1.873139
BFGS:    2 13:27:45      -15.037950        0.158560
BFGS:    3 13:27:45      -15.038231        0.012308
BFGS:    4 13:27:46      -15.038233        0.000099
Final lattice:
 Cell([3.6108030363492873, 3.6108010096864436, 3.610803010703384])
Final positions:
 [[ 3.00438182e-10  4.98521716e-10  3.22463221e-10]
 [ 7.88471858e-10  1.80540050e+00  1.80540151e+00]
 [ 1.80540152e+00 -6.04277892e-11  1.80540151e+00]
 [ 1.80540152e+00  1.80540050e+00  2.67322374e-10]]


In [ ]:
from ase.build import bulk
atoms = bulk("Cu", "fcc", a = 3.6, cubic = True)
from ase.visualize import view
view(atoms, viewer = "x3d")

In [ ]:
from ase.io import read, write
def relax_system(filename):
  atoms = read(f"{filename}.POSCAR")
  atoms.calc = calc
  opt1 = BFGS(atoms)
  opt1.run(fmax = 0.01)
  cell_filter = ExpCellFilter(atoms, mask=[1, 1, 1, 0, 0, 0])
  optimizer = BFGS(cell_filter)
  optimizer.run(fmax = 0.01)
  write(f"{filename}_opt.POSCAR", atoms)

filenames = ["Cu110", "Cu100", "Cu211", "Cu111"]
for filename in filenames:
  print(f"Started {filename} relaxation")
  relax_system(filename)
  print(f"Completed {filename} relxation")

Started Cu110 relaxation
      Step     Time          Energy          fmax
BFGS:    0 13:29:44     -209.527753        0.395592
BFGS:    1 13:29:48     -209.646461        0.360655
BFGS:    2 13:29:52     -210.356681        0.070024
BFGS:    3 13:29:56     -210.360744        0.064571
BFGS:    4 13:30:00     -210.378061        0.046668
BFGS:    5 13:30:04     -210.384395        0.019685
BFGS:    6 13:30:08     -210.386053        0.004973
      Step     Time          Energy          fmax
BFGS:    0 13:30:08     -210.386053       16.077934


<ipython-input-5-349862757>:7: FutureWarning: Import ExpCellFilter from ase.filters
  cell_filter = ExpCellFilter(atoms, mask=[1, 1, 1, 0, 0, 0])


BFGS:    1 13:30:15     -115.707334      752.773135
BFGS:    2 13:30:19     -210.770162        7.313535
BFGS:    3 13:30:23     -209.899543       39.355862
BFGS:    4 13:30:28     -212.259809       52.101100
BFGS:    5 13:30:33     -215.568222       47.991300
BFGS:    6 13:30:39     -216.427282       38.017551
BFGS:    7 13:30:45     -216.943651       37.983432
BFGS:    8 13:30:51     -217.413572       38.598520
BFGS:    9 13:30:57     -217.678550       40.130312
BFGS:   10 13:31:03     -218.880880       42.252347
BFGS:   11 13:31:08     -219.775510       30.010512
BFGS:   12 13:31:13     -220.168674        8.194306
BFGS:   13 13:31:19     -220.222919        4.073298
BFGS:   14 13:31:24     -220.235710        2.725921
BFGS:   15 13:31:29     -220.238830        0.526411
BFGS:   16 13:31:34     -220.239036        0.163479
BFGS:   17 13:31:39     -220.239028        0.108837
BFGS:   18 13:31:44     -220.239048        0.032494
BFGS:   19 13:31:49     -220.239044        0.015290
BFGS:   20 1

In [ ]:
predictor1 = pretrained_mlip.get_predict_unit("uma-s-1", device="cpu")
calc1 = FAIRChemCalculator(predictor1, task_name="oc20")

def relax_system(filename):
  atoms = read(f"{filename}.POSCAR")
  atoms.calc = calc1
  opt1 = BFGS(atoms)
  opt1.run(fmax = 0.01)
  cell_filter = ExpCellFilter(atoms, mask=[1, 1, 1, 0, 0, 0])
  optimizer = BFGS(cell_filter)
  optimizer.run(fmax = 0.01)
  write(f"{filename}_opt.POSCAR", atoms)

filenames = ["Cu110", "Cu100", "Cu211", "Cu111"]
for filename in filenames:
  print(f"Started {filename} relaxation")
  relax_system(filename)
  print(f"Completed {filename} relaxation")

Started Cu110 relaxation
      Step     Time          Energy          fmax
BFGS:    0 13:45:04     -183.552782        0.439572
BFGS:    1 13:45:08     -183.637442        0.397155
BFGS:    2 13:45:11     -184.021678        0.022903
BFGS:    3 13:45:15     -184.023568        0.011441
BFGS:    4 13:45:19     -184.024233        0.007392
      Step     Time          Energy          fmax
BFGS:    0 13:45:19     -184.024233        8.261858


<ipython-input-6-2071020841>:9: FutureWarning: Import ExpCellFilter from ase.filters
  cell_filter = ExpCellFilter(atoms, mask=[1, 1, 1, 0, 0, 0])


BFGS:    1 13:45:23     -181.476955       60.178467
BFGS:    2 13:45:27     -185.785278       26.991784
BFGS:    3 13:45:32     -137.062481      568.698656
BFGS:    4 13:45:37     -190.524237       36.482320
BFGS:    5 13:45:42     -177.455160      231.948306
BFGS:    6 13:45:47     -192.002067       10.598505
BFGS:    7 13:45:52     -192.272002        8.582943
BFGS:    8 13:45:57     -192.435619        4.579160
BFGS:    9 13:46:01     -192.451298        2.954906
BFGS:   10 13:46:07     -192.455028        1.947362
BFGS:   11 13:46:11     -192.457373        0.611667
BFGS:   12 13:46:17     -192.457611        0.313241
BFGS:   13 13:46:21     -192.457709        0.065855
BFGS:   14 13:46:26     -192.457727        0.037401
BFGS:   15 13:46:31     -192.457726        0.013785
BFGS:   16 13:46:36     -192.457727        0.001760
Completed Cu110 relaxation
Started Cu100 relaxation
      Step     Time          Energy          fmax
BFGS:    0 13:46:41     -192.154301        0.247767
BFGS:    1 13: